In [3]:
import pandas as pd
import numpy as np
import string
import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import os
import keras_nlp
import tensorflow as tf
import keras_core as keras
from transformers import pipeline, set_seed, AutoTokenizer

In [4]:
# Set display options
pd.set_option('display.max_colwidth', None)

In [5]:
# Load the dataset
df = pd.read_csv("datasets/NLP/companies_data.csv")

In [12]:
# Define functions for data preprocessing
def remove_punctuation(text):
    punctuationFree = "".join([i for i in text if i not in string.punctuation])
    return punctuationFree

def tokenization(text):
    tokens = re.split("\W+", text)
    return tokens

def remove_stopwords(text):
    output = [i for i in text if i not in stopWords]
    return output

def stemming(text):
    porter_stemmer = PorterStemmer()
    stem_text = [porter_stemmer.stem(word) for word in text]
    return stem_text

def lemmatizer(text):
    wordnet_lemmatizer = WordNetLemmatizer()
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text

In [10]:
# Download NLTK resources
# nltk.download()

# Load NLTK stop words
stopWords = nltk.corpus.stopwords.words('english')

In [13]:
# Apply preprocessing steps to the DataFrame
df["puncFree"] = df['description'].apply(lambda x: remove_punctuation(x))
df["lowercased"] = df["puncFree"].apply(lambda x: x.lower())
df["tokenized"] = df["lowercased"].apply(lambda x: tokenization(x))
df["no_stop_words"] = df["tokenized"].apply(lambda x: remove_stopwords(x))
df['stemmed'] = df['no_stop_words'].apply(lambda x: stemming(x))
df['lemmatized'] = df['no_stop_words'].apply(lambda x: lemmatizer(x))

In [14]:
# Load Keras NLP models and preprocessors
preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset("gpt2_base_en", sequence_length=128)
gpt2_lm = keras_nlp.models.GPT2CausalLM.from_preset("gpt2_base_en", preprocessor=None)

2023-08-18 17:11:02.371699: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [15]:
# Load Hugging Face's GPT-2 tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

Using pad_token, but it is not set yet.


In [16]:
# Tokenize the data
tokenized_data_main = tokenizer([str(i) for i in df["lemmatized"].values], return_tensors="np", padding=True)
tokenized_data_label = tokenizer([str(i) for i in df["companyName"].values], return_tensors="np", padding=True)


In [17]:
# Prepare tokenized data
tokenized_data_main = dict(tokenized_data_main)
tokenized_data_label = dict(tokenized_data_label)
tokenized_data_main['padding_mask'] = tokenized_data_main['attention_mask']
del tokenized_data_main['attention_mask']
tokenized_data_main['token_ids'] = tokenized_data_main['input_ids']
del tokenized_data_main['input_ids']
tokenized_data_label['padding_mask'] = tokenized_data_label['attention_mask']
del tokenized_data_label['attention_mask']
tokenized_data_label['token_ids'] = tokenized_data_label['input_ids']
del tokenized_data_label['input_ids']

In [19]:
# Define training data
x = tokenized_data_main
y = tokenized_data_label
num_epochs = 1

# Compile and fit the model
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
gpt2_lm.compile(
    optimizer = Adam(learning_rate=0.001),
    loss=loss,
    weighted_metrics=["accuracy"],
)
gpt2_lm.fit(x=x, y=y, epochs=num_epochs)

NameError: name 'Adam' is not defined

In [21]:
import pandas as pd
import numpy as np
import string
import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import os
import keras_nlp
import tensorflow as tf
from transformers import AutoTokenizer
from keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy
import keras

# Set display options
pd.set_option('display.max_colwidth', None)

# Load the dataset
df = pd.read_csv("datasets/NLP/companies_data.csv")

# Define functions for data preprocessing
def remove_punctuation(text):
    punctuationFree = "".join([i for i in text if i not in string.punctuation])
    return punctuationFree

def tokenization(text):
    tokens = re.split("\W+", text)
    return tokens

def remove_stopwords(text):
    output = [i for i in text if i not in stopWords]
    return output

def stemming(text):
    porter_stemmer = PorterStemmer()
    stem_text = [porter_stemmer.stem(word) for word in text]
    return stem_text

def lemmatizer(text):
    wordnet_lemmatizer = WordNetLemmatizer()
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text

# Download NLTK resources
nltk.download()

# Load NLTK stop words
stopWords = nltk.corpus.stopwords.words('english')

# Apply preprocessing steps to the DataFrame
df["puncFree"] = df['description'].apply(lambda x: remove_punctuation(x))
df["lowercased"] = df["puncFree"].apply(lambda x: x.lower())
df["tokenized"] = df["lowercased"].apply(lambda x: tokenization(x))
df["no_stop_words"] = df["tokenized"].apply(lambda x: remove_stopwords(x))
df['stemmed'] = df['no_stop_words'].apply(lambda x: stemming(x))
df['lemmatized'] = df['no_stop_words'].apply(lambda x: lemmatizer(x))

# Load Keras NLP models and preprocessors
preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset("gpt2_base_en", sequence_length=128)
gpt2_lm = keras_nlp.models.GPT2CausalLM.from_preset("gpt2_base_en", preprocessor=None)

# Load Hugging Face's GPT-2 tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Tokenize the data
tokenized_data_main = tokenizer([str(i) for i in df["lemmatized"].values], return_tensors="np", padding=True)
tokenized_data_label = tokenizer([str(i) for i in df["companyName"].values], return_tensors="np", padding=True)

# ... Rest of your code ...

# Compile and fit the model
loss = SparseCategoricalCrossentropy(from_logits=True)
optimizer = Adam(learning_rate=0.001)  # Adjust the learning rate as needed
gpt2_lm.compile(
    optimizer=optimizer,
    loss=loss,
    weighted_metrics=["accuracy"],
)
gpt2_lm.fit(x=x, y=y, epochs=num_epochs)


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


Using pad_token, but it is not set yet.
2023-08-18 17:15:00.760067: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [190,15]
	 [[{{node Placeholder/_3}}]]


ValueError: in user code:

    File "/home/wondermary/anaconda3/envs/NLP2/lib/python3.11/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/home/wondermary/anaconda3/envs/NLP2/lib/python3.11/site-packages/keras/engine/training.py", line 1249, in run_step  *
        outputs = model.train_step(data)
    File "/home/wondermary/anaconda3/envs/NLP2/lib/python3.11/site-packages/keras/engine/training.py", line 1051, in train_step  **
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/wondermary/anaconda3/envs/NLP2/lib/python3.11/site-packages/keras/engine/training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "/home/wondermary/anaconda3/envs/NLP2/lib/python3.11/site-packages/keras/engine/compile_utils.py", line 236, in __call__
        y_true = self._conform_to_outputs(y_pred, y_true)
    File "/home/wondermary/anaconda3/envs/NLP2/lib/python3.11/site-packages/